In [1]:
# --------------------------------------------------------
# Overview 
# --------------------------------------------------------
# Objective is to predictive output for pm, stator tooth, stator winding
# and stator yoke.  The key measure is RMSE
#  judging criteria
#  1. Code structure/quality
#  2. Data mining
#  3. Findings and explanations
#  4. Predictions and performance of the model 

In [1]:
import pandas as pd
from pycaret.regression import *
import sklearn
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn import model_selection
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split

In [6]:
# Load data
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
test_solutions = pd.read_csv("test_solutions.csv")
test = pd.concat([test, test_solutions],axis=1)

In [3]:
train.head()

,ambient,coolant,u_d,u_q,motor_speed,torque,i_d,i_q,pm,stator_yoke,stator_tooth,stator_winding,profile_id
0,-0.752143,-1.118446,0.327935,-1.297858,-1.222428,-0.250182,1.029572,-0.245860,-2.522071,-1.831422,-2.066143,-2.018033,4
1,-0.771263,-1.117021,0.329665,-1.297686,-1.222429,-0.249133,1.029509,-0.245832,-2.522418,-1.830969,-2.064859,-2.017631,4
2,-0.782892,-1.116681,0.332771,-1.301822,-1.222428,-0.249431,1.029448,-0.245818,-2.522673,-1.830400,-2.064073,-2.017343,4
3,-0.780935,-1.116764,0.333700,-1.301852,-1.222430,-0.248636,1.032845,-0.246955,-2.521639,-1.830333,-2.063137,-2.017632,4
4,-0.774043,-1.116775,0.335206,-1.303118,-1.222429,-0.248701,1.031807,-0.246610,-2.521900,-1.830498,-2.062795,-2.018145,4


In [5]:
test_solutions.head()

,pm,stator_tooth,stator_yoke,stator_winding
0,-2.450515,-2.046755,-1.800853,-1.993896
1,-2.450277,-2.047172,-1.801808,-1.994415
2,-2.450026,-2.047384,-1.802578,-1.995002
3,-2.449813,-2.047343,-1.801862,-1.995174
4,-2.450108,-2.047665,-1.801228,-1.995189


In [7]:
test.head()

,ambient,coolant,u_d,u_q,motor_speed,torque,i_d,i_q,pm,stator_tooth,stator_yoke,stator_winding
0,-2.749564,-1.073478,0.336463,-1.295874,-1.222426,-0.248547,1.041590,-0.250223,-2.450515,-2.046755,-1.800853,-1.993896
1,-2.749612,-1.072823,0.339015,-1.298281,-1.222425,-0.247740,1.038055,-0.248936,-2.450277,-2.047172,-1.801808,-1.994415
2,-2.746339,-1.073277,0.340775,-1.299160,-1.222428,-0.247232,1.035532,-0.248042,-2.450026,-2.047384,-1.802578,-1.995002
3,-2.747337,-1.074854,0.341972,-1.300307,-1.222428,-0.246810,1.033727,-0.247364,-2.449813,-2.047343,-1.801862,-1.995174
4,-2.748052,-1.076696,0.344047,-1.301804,-1.222429,-0.246473,1.032427,-0.246897,-2.450108,-2.047665,-1.801228,-1.995189


In [8]:
main_columns = ['ambient','coolant','u_d','u_q','motor_speed','torque','i_d','i_q']

In [11]:
train[main_columns]

,ambient,coolant,u_d,u_q,motor_speed,torque,i_d,i_q
0,-0.752143,-1.118446,0.327935,-1.297858,-1.222428,-0.250182,1.029572,-0.245860
1,-0.771263,-1.117021,0.329665,-1.297686,-1.222429,-0.249133,1.029509,-0.245832
2,-0.782892,-1.116681,0.332771,-1.301822,-1.222428,-0.249431,1.029448,-0.245818
3,-0.780935,-1.116764,0.333700,-1.301852,-1.222430,-0.248636,1.032845,-0.246955
4,-0.774043,-1.116775,0.335206,-1.303118,-1.222429,-0.248701,1.031807,-0.246610
...,...,...,...,...,...,...,...,...
846363,-0.047497,0.341638,0.331475,-1.246114,-1.222428,-0.255640,1.029142,-0.245722
846364,-0.048839,0.320022,0.331701,-1.250655,-1.222437,-0.255640,1.029148,-0.245736
846365,-0.042350,0.307415,0.330946,-1.246852,-1.222430,-0.255640,1.029191,-0.245701
846366,-0.039433,0.302082,0.330987,-1.249505,-1.222432,-0.255640,1.029147,-0.245727


In [13]:
# rescale all variables except the target variable
scaler = preprocessing.MinMaxScaler()
df_scale = scaler.fit_transform(train[main_columns])
df_scale = pd.DataFrame(df_scale)
df_scale.reset_index(drop=True, inplace=True)

In [ ]:
# combine rescaled value
df['disease'].reset_index(drop=True, inplace=True)
df_1 = pd.concat([df_scale,df['disease']], axis=1)
df_1 = df.columns.values.tolist()
df.shape

In [ ]:
# combine rescaled value
df['disease'].reset_index(drop=True, inplace=True)
df_1 = pd.concat([df_scale,df['disease']], axis=1)
df_1 = df.columns.values.tolist()
df.shape

In [ ]:
exp_class101 = setup(data = train, target = 'disease', session_id=123) 

In [ ]:
# build pm model

# build stator tooth model


# build stator yoke model

# build startor winding model

In [ ]:
# =======================================================
# outputs
# =======================================================
output_cols <- c('pm','stator_tooth','stator_yoke','stator_winding')
rmse_col <- c('RMSE_pm', 'RMSE_stator_yoke', 'RMSE_stator_tooth', 'RMSE_stator_winding')

#predictions
y_pred_combined <- cbind(y_pred_pm,y_pred_stator_tooth,y_pred_stator_yoke,y_pred_stator_winding)
y_pred_combined <- as.data.frame(y_pred_combined)
colnames(y_pred_combined) <- output_cols
write.csv(y_pred_combined, "predictions.csv")

#rmse
rmse <- function(error)
{
  sqrt(mean(error^2))
}
error_pm <- Target_test_pm -  y_pred_pm
error_stator_tooth <- Target_train_stator_tooth - y_pred_model_stator_tooth
error_stator_yoke <- Target_test_stator_yoke -  y_pred_stator_yoke
error_stator_winding <- Target_test_stator_winding - y_pred_stator_winding

rmse_pm <- rmse(error_pm)
rmse_stator_tooth <- rmse(error_stator_tooth)
rmse_stator_yoke <- rmse(error_stator_yoke)
rmse_stator_winding <- rmse(error_stator_winding)

rmse_total <- cbind(rmse_pm, rmse_stator_tooth, rmse_stator_yoke, rmse_stator_winding)
rmse_total <- as.data.frame(rmse_total)
colnames(rmse_total) <- rmse_col

In [ ]:


rms = mean_squared_error(y_actual, y_predicted, squared=False)